In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Connect to CRUD module
from CRUD_Python_Module import AnimalShelter

# Connection credentials
username = "aacuser"
password = "SNHU1234"

# Connect to database
db = AnimalShelter(username, password)

# Load data
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Queries
water_query = {
    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
    "age_upon_outcome_in_weeks": {"$lte": 104}
}
mountain_query = {
    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog"]},
    "age_upon_outcome_in_weeks": {"$lte": 156}
}
disaster_query = {
    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Bloodhound"]},
    "age_upon_outcome_in_weeks": {"$lte": 156}
}

# App layout
app = JupyterDash(__name__)

# Logo setup
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '150px'}),
        html.H2('Dashboard by Mario Frederick')
    ]),
    html.Hr(),

    # Radio buttons
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        )
    ]),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        page_size=10
    ),
    html.Br(),
    html.Hr(),

    # Charts and map side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'height': '500px'}  # ✅ ensure the map container has height
        )
    ])
])

######################################
# Callbacks
######################################

# Update Data Table
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        results = db.read(water_query)
    elif filter_type == 'mountain':
        results = db.read(mountain_query)
    elif filter_type == 'disaster':
        results = db.read(disaster_query)
    else:
        results = db.read({})

    filtered_df = pd.DataFrame.from_records(results)
    filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

# Pie chart
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data to display")]

    dff = pd.DataFrame(viewData)
    return [
        dcc.Graph(figure=px.pie(dff, names='breed', title='Preferred Animals'))
    ]

# Highlight column
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Geolocation map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or index is None or len(index) == 0:
        return [html.P("Select a row to view location")]

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    # Debug logs
    print("Selected row index:", row)
    print(dff.iloc[row][['name', 'location_lat', 'location_long']])

    latitude = float(dff.iloc[row]['location_lat'])
    longitude = float(dff.iloc[row]['location_long'])
    print("Lat/Lon:", latitude, longitude)

    return [dl.Map(center=[latitude, longitude], zoom=10,
                   style={'width': '600px', 'height': '400px', 'border': '2px solid red'},  # ✅ border for debugging
                   children=[
                       dl.TileLayer(),
                       dl.Marker(position=[latitude, longitude], children=[
                           dl.Tooltip(dff.iloc[row]['breed']),
                           dl.Popup([
                               html.H4("Animal"),
                               html.P(dff.iloc[row]['name'])
                           ])
                       ])
                   ])]

# Run app
app.run_server()

Connected to MongoDB successfully!
Dash app running on https://pedroaloha-horizonparadox-3000.codio.io/proxy/8050/
Selected row index: 1
name                  Zeus
location_lat     30.636043
location_long   -97.420354
Name: 1, dtype: object
Lat/Lon: 30.6360434662318 -97.4203536438348
